https://blog.csdn.net/weixin_39653948/article/details/105385622

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Flatten, Bidirectional
from tensorflow.keras.layers import TimeDistributed, RepeatVector

In [2]:
class MultiStepModels:
    '''
    多時間步 預測 時間序列 LSTM 模型
    '''

    def __init__(self, train_seq, test_seq, sw_width, pred_length,
                 features, epochs_num, verbose_set, flag = 0):
        self.train_seq = train_seq
        self.test_seq = test_seq
        self.sw_width = sw_width
        self.pred_length = pred_length

        self.features = features
        self.epochs_num = epochs_num

        # verbose = 0 为不在标准输出流输出日志信息
        # verbose = 1 为输出进度条记录
        # verbose = 2 为每个epoch输出一行记录
        self.verbose_set = verbose_set

        self.flag = flag

        self.X, self.y = [], []

    def split_sequence(self):
        '''
        該 函數 實現 多輸入序列數據的樣本劃分
        '''

        for i in range(len(self.train_seq)):
            # 找到最後一個元素的索引，因為for循環中i從1開始，切片索引從0開始，切片區間前閉後開，所以不用減去1;
            end_index = i + self.sw_width

            # 找到須要預測指定時間步長的最後一個元素之索引;
            out_end_index = end_index + self.pred_length

            # 如果最後一個期望 輸出最後一個元素的索引 大於 序列中最後一元素的索引則丟棄該樣本;
            # 這裡len(self.sequence)沒有減去1的原因是：保證最後一個元素的索引剛好等於序列數據索引的時候，能夠擷取到該樣本;
            if out_end_index > len(self.train_seq):
                break

            # 實現以 滑動步伐 為 1，窗口寬度為 self.sw_width 的滑動步伐 取值。
            seq_x, seq_y = self.train_seq[i:end_index], self.train_seq[end_index:out_end_index]
            self.X.append(seq_x)
            self.y.append(seq_y)

        self.X, self.y = np.array(self.X), np.array(self.y)
        self.X = self.X.reshape((self.X.shape[0], self.X.shape[1], self.features))
        self.test_seq = self.test_seq.reshape((1, self.sw_width, self.features))

        # 什麼意思？
        if self.flag == 1:
            self.y = self.y.reshape((self.y.shape[0], self.y.shape[1], self.features))
        else:
            pass

        for i in range(len(self.X)):
            print(self.X[i], self.y[i])


        print('X:\n{} \n\ny:\n{} \n\ntest_seq:\n{} \n'.format(self.X, self.y, self.test_seq))
        print('X.shape:{}, y.shape:{}, test_seq.shape:{}\n'.format(self.X.shape, self.y.shape, self.test_seq.shape))

        return self.X, self.y, self.test_seq

    def stacked_lstm(self):
        print(f'X.shpae: {self.X.shape}')
        print(f'y.shape: {self.y.shape}')
        model = Sequential()
        model.add(LSTM(100, activation='relu', return_sequences=True,
                       input_shape = (self.sw_width, self.features)))
        model.add(LSTM(100, activation='relu'))
        model.add(Dense(units=self.pred_length))
        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
        print(model.summary())

        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']),
              '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('y^hat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')

    def encoder_decoder_lstm(self):
        model = Sequential()
        model.add(LSTM(100, activation='relu',
                       input_shape = (self.sw_width, self.features)))
        model.add(RepeatVector(self.pred_length))
        model.add(LSTM(100, activation='relu', return_sequences=True))
        model.add(TimeDistributed(Dense(1)))

        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
        print(model.summary())

        history = model.fit(self.X, self.y, epochs=self.epochs_num, verbose=self.verbose_set)
        print('\ntrain_acc:%s'%np.mean(history.history['accuracy']),
              '\ntrain_loss:%s'%np.mean(history.history['loss']))
        print('y^hat:%s'%(model.predict(self.test_seq)),'\n-----------------------------')

In [3]:
# import pandas as pd
#
# dataframe = pd.read_csv('data_processed/Ba_Leng/2019_Ba_Leng_Rainfall.csv')
# dataframe['Time'] = pd.to_datetime(dataframe['Time'])
# dataframe

In [4]:
# filter = (dataframe["Month"] == 8)
# by_month = dataframe[filter]
# by_month

In [5]:
# import matplotlib.pyplot as plt
# start, end = 170, 210
# x_axis_data = by_month[["Time"]][start:end]
#
# y_axis_data = by_month[["Rainfall"]][start:end]
#
# plt.plot(x_axis_data, y_axis_data)
# plt.show()

In [6]:
# train_seq = np.array(y_axis_data[0:28])
# test_seq = np.array(y_axis_data[21:28])
# train_seq = np.squeeze(train_seq)
# test_seq = np.squeeze(test_seq)

In [ ]:
if __name__ == '__main__':

    train_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    test_seq = np.array([70, 80, 90])

    sliding_window_width = 3
    predict_length = 2
    n_features = 1

    epoch_num = 1000
    verbose_set = 0

    print('-------以下為【向量輸出 LSTM 模型 】相關訊息-------')
    MultiStepLSTM = MultiStepModels(train_seq, test_seq, sliding_window_width, predict_length,
                                    n_features, epoch_num, verbose_set)
    MultiStepLSTM.split_sequence()
    MultiStepLSTM.stacked_lstm()

    print('-------以下為【編碼器-解碼器 LSTM 模型 】相關訊息-------')
    MultiStepLSTM = MultiStepModels(train_seq, test_seq, sliding_window_width, predict_length,
                                    n_features, epoch_num, verbose_set, flag=1)
    MultiStepLSTM.split_sequence()
    MultiStepLSTM.encoder_decoder_lstm()


-------以下為【向量輸出 LSTM 模型 】相關訊息-------
[[10]
 [20]
 [30]] [40 50]
[[20]
 [30]
 [40]] [50 60]
[[30]
 [40]
 [50]] [60 70]
[[40]
 [50]
 [60]] [70 80]
[[50]
 [60]
 [70]] [80 90]
X:
[[[10]
  [20]
  [30]]

 [[20]
  [30]
  [40]]

 [[30]
  [40]
  [50]]

 [[40]
  [50]
  [60]]

 [[50]
  [60]
  [70]]] 

y:
[[40 50]
 [50 60]
 [60 70]
 [70 80]
 [80 90]] 

test_seq:
[[[70]
  [80]
  [90]]] 

X.shape:(5, 3, 1), y.shape:(5, 2), test_seq.shape:(1, 3, 1)

X.shpae: (5, 3, 1)
y.shape: (5, 2)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3, 100)            40800     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 2)                 202       
                                            